CAP5610 HW5 - Yuan Du

Code stored at my Github: https://github.com/YuanEldaif/CAP5610

In [1]:
#Data analysis and wrangling
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import math
import random
import time
from tkinter import *

In [2]:

#https://stackoverflow.com/questions/7498658/importerror-when-importing-tkinter-in-python

######################################################################
# This section contains functions for loading CSV (comma separated values)
# files and convert them to a dataset of instances.
# Each instance is a tuple of attributes. The entire dataset is a list
# of tuples.
######################################################################

# Loads a CSV files into a list of tuples.
# Ignores the first row of the file (header).
# Numeric attributes are converted to floats, nominal attributes
# are represented with strings.
# Parameters:
#   fileName: name of the CSV file to be read
# Returns: a list of tuples
def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0] # remove the header
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset

# Converts a comma separated string into a tuple
# Parameters
#   line: a string
# Returns: a tuple
def lineToTuple(line):
    # remove leading/trailing witespace and newlines
    cleanLine = line.strip()
    # get rid of quotes
    cleanLine = cleanLine.replace('"', '')
    # separate the fields
    lineList = cleanLine.split(",")
    # convert strings into numbers
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    return lineTuple

# Destructively converts all the string elements representing numbers
# to floating point numbers.
# Parameters:
#   myList: a list of strings
# Returns None
def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

# Checks if a given string can be safely converted into a positive float.
# Parameters:
#   s: the string to be checked
# Returns: True if the string represents a positive float, False otherwise
def isValidNumberString(s):
  if len(s) == 0:
    return False
  if  len(s) > 1 and s[0] == "-":
      s = s[1:]
  for c in s:
    if c not in "0123456789.":
      return False
  return True


In [3]:
#https://github.com/mohammedjasam/K-Means-Clustering

######################################################################
# This section contains functions for clustering a dataset
# using the k-means algorithm.
######################################################################

def distance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    sumOfSquares = 0
    for i in range(1, len(instance1)):
        sumOfSquares += (instance1[i] - instance2[i])**2
    return sumOfSquares

def Mdistance(instance1, instance2):
    if instance1 == None or instance2 == None:
        return float("inf")
    Manhattan = 0
    for i in range(1, len(instance1)):
        Manhattan += abs(instance1[i]-instance2[i])
    return Manhattan

#def mdistance(instance1, instance2):
#return sum(abs(a-b) for a,b in zip(x,y))

from scipy import spatial
def cdistance(x, y):
    from math import sqrt
    def dot_product(x, y):
        return sum(a * b for a, b in zip(x, y))
    def magnitude(vector):
        return sqrt(dot_product(vector, vector))
    def similarity(x, y):
        return (1-(dot_product(x, y) / (magnitude(x) * magnitude(y) + .00000000001)))
#def cdistance(x,y)
        #create normalized copies of each vector:
#    x_norm = np.linalg.norm(x)
#    y_norm = np.linalg.norm(y)
    #compute cosine similarity:
#    sim = np.dot(x, y)/(x_norm * y_norm) #this should return a scalar float
#    return sim
    
def jdistance(a,b):
    c = set(a).intersection(set(b))
    return (1- float(len(c)) / (len(a) + len(b) - len(c)))


def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes-1)
    for instance in instanceList:
        for i in range(1, numAttributes):
            means[i] += instance[i]
    for i in range(1, numAttributes):
        means[i] /= float(numInstances)
    return tuple(means)

def assign(instance, centroids):
    minDistance = distance(instance, centroids[0])
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = distance(instance, centroids[i])
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def Massign(instance, centroids):
    minDistance = Mdistance(instance, centroids[0])
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = Mdistance(instance, centroids[i])
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def cassign(instance, centroids):
    minDistance = cdistance(instance, centroids[0])
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = cdistance(instance, centroids[i])
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def jassign(instance, centroids):
    minDistance = jdistance(instance, centroids[0])
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = jdistance(instance, centroids[i])
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

def assignAll(instances, centroids):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = assign(instance, centroids)
        clusters[clusterIndex].append(instance)
    return clusters

def MassignAll(instances, centroids):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = Massign(instance, centroids)
        clusters[clusterIndex].append(instance)
    return clusters

def cassignAll(instances, centroids):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = cassign(instance, centroids)
        clusters[clusterIndex].append(instance)
    return clusters

def jassignAll(instances, centroids):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = jassign(instance, centroids)
        clusters[clusterIndex].append(instance)
    return clusters

def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "centroid" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

#Euclidean Similarity
def kmeans(instances, k, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        canvas = prepareWindow(instances)
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        paintClusters2D(canvas, clusters, centroids, "Initial centroids")
        time.sleep(delay)
    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        clusters = assignAll(instances, centroids)
        if animation:
            paintClusters2D(canvas, clusters, centroids, "Assign %d" % iteration)
            time.sleep(delay)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids)
        if animation:
            paintClusters2D(canvas, clusters, centroids,
                            "Update %d, withinss %.1f" % (iteration, withinss))
            time.sleep(delay)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["sse"]=withinss
    result["iteration"] = iteration
    return result

def computeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += distance(centroid, instance)
    return result

def McomputeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += Mdistance(centroid, instance)
    return result

def ccomputeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += cdistance(centroid, instance)
    return result

def jcomputeWithinss(clusters, centroids):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += jdistance(centroid, instance)
    return result

# Repeats k-means clustering n times, and returns the clustering
# with the smallest withinss
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i ,
        trialClustering = kmeans(instances, k))
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print ("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering

In [4]:
######################################################################
# This section contains functions for visualizing datasets and
# clustered datasets.
######################################################################

def printTable(instances):
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print (line)

def extractAttribute(instances, index):
    result = []
    for instance in instances:
        result.append(instance[index])
    return result

def paintCircle(canvas, xc, yc, r, color):
    canvas.create_oval(xc-r, yc-r, xc+r, yc+r, outline=color)

def paintSquare(canvas, xc, yc, r, color):
    canvas.create_rectangle(xc-r, yc-r, xc+r, yc+r, fill=color)

def drawPoints(canvas, instances, color, shape):
    random.seed(0)
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for instance in instances:
        x = 5*(random.random()-0.5)+margin+(instance[1]-minX)*scaleX
        y = 5*(random.random()-0.5)+height-margin-(instance[2]-minY)*scaleY
        if (shape == "square"):
            paintSquare(canvas, x, y, 5, color)
        else:
            paintCircle(canvas, x, y, 5, color)
    canvas.update()

def connectPoints(canvas, instances1, instances2, color):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for p1 in instances1:
        for p2 in instances2:
            x1 = margin + (p1[1]-minX)*scaleX
            y1 = height - margin - (p1[2]-minY)*scaleY
            x2 = margin + (p2[1]-minX)*scaleX
            y2 = height - margin - (p2[2]-minY)*scaleY
            canvas.create_line(x1, y1, x2, y2, fill=color)
    canvas.update()

def mergeClusters(clusters):
    result = []
    for cluster in clusters:
        result.extend(cluster)
    return result

def prepareWindow(instances):
    width = 500
    height = 500
    margin = 50
    root = Tk()
    canvas = Canvas(root, width=width, height=height, background="white")
    canvas.pack()
    canvas.data = {}
    canvas.data["margin"] = margin
    setBounds2D(canvas, instances)
    paintAxes(canvas)
    canvas.update()
    return canvas

def setBounds2D(canvas, instances):
    attributeX = extractAttribute(instances, 1)
    attributeY = extractAttribute(instances, 2)
    canvas.data["minX"] = min(attributeX)
    canvas.data["minY"] = min(attributeY)
    canvas.data["maxX"] = max(attributeX)
    canvas.data["maxY"] = max(attributeY)

def paintAxes(canvas):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    canvas.create_line(margin/2, height-margin/2, width-5, height-margin/2,
                       width=2, arrow=LAST)
    canvas.create_text(margin, height-margin/4,
                       text=str(minX), font="Sans 11")
    canvas.create_text(width-margin, height-margin/4,
                       text=str(maxX), font="Sans 11")
    canvas.create_line(margin/2, height-margin/2, margin/2, 5,
                       width=2, arrow=LAST)
    canvas.create_text(margin/4, height-margin,
                       text=str(minY), font="Sans 11", anchor=W)
    canvas.create_text(margin/4, margin,
                       text=str(maxY), font="Sans 11", anchor=W)
    canvas.update()


def showDataset2D(instances):
    canvas = prepareWindow(instances)
    paintDataset2D(canvas, instances)

def paintDataset2D(canvas, instances):
    canvas.delete(ALL)
    paintAxes(canvas)
    drawPoints(canvas, instances, "blue", "circle")
    canvas.update()

def showClusters2D(clusteringDictionary):
    clusters = clusteringDictionary["clusters"]
    centroids = clusteringDictionary["centroids"]
    withinss = clusteringDictionary["withinss"]
    canvas = prepareWindow(mergeClusters(clusters))
    paintClusters2D(canvas, clusters, centroids,
                    "Withinss: %.1f" % withinss)

def paintClusters2D(canvas, clusters, centroids, title=""):
    canvas.delete(ALL)
    paintAxes(canvas)
    colors = ["blue", "red", "green", "brown", "purple", "orange"]
    for clusterIndex in range(len(clusters)):
        color = colors[clusterIndex%len(colors)]
        instances = clusters[clusterIndex]
        centroid = centroids[clusterIndex]
        drawPoints(canvas, instances, color, "circle")
        if (centroid != None):
            drawPoints(canvas, [centroid], color, "square")
        connectPoints(canvas, [centroid], instances, color)
    width = canvas.winfo_reqwidth()
    canvas.create_text(width/2, 20, text=title, font="Sans 14")
    canvas.update()


In [5]:
######################################################################
# Test code
######################################################################

dataset = loadCSV("C:/Work/Project/CAP5610/HW/HW5/Data.csv")
showDataset2D(dataset)

## Task 1: Suppose we have 10 college football teams X1 to X10. We want to cluster them into 2 groups. For each football team, we have two features: One is # wins in Season 2016, and the other is # wins in Season 2017.
### (1) Initialize with two centroids, (4, 6) and (5, 4). Use Manhattan distance as the distance metric. First, perform one iteration of the K-means algorithm and report the coordinates of the resulting centroids. Second, please use K-Means to find two clusters.



In [6]:
#Manhattan Kmeans
#maxiter: control number of iteration
def Mkmeans(instances, k, maxiter, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        canvas = prepareWindow(instances)
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        paintClusters2D(canvas, clusters, centroids, "Initial centroids")
        time.sleep(delay)
    for itr in range(maxiter):
    #iteration = 0
    #while (centroids != prevCentroids):
        #iteration += 1
        clusters = MassignAll(instances, centroids)
        
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = McomputeWithinss(clusters, centroids)
        
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["sse"]=withinss
    return result


In [7]:
initCentroids1 = [('',4,6),('',5,4)]
clustering = Mkmeans(dataset, 2, 1,animation=False, initCentroids=initCentroids1)
c=clustering["clusters"]
printTable(clustering["centroids"])
print('Xs assigned to two clusters are:','- cluster 1:', c[0], '- cluster 2:', c[1])

centroid0	4.00 6.33 
centroid1	5.57 3.57 
Xs assigned to two clusters are: - cluster 1: [('X1', 3.0, 5.0), ('X3', 2.0, 8.0), ('X10', 7.0, 6.0)] - cluster 2: [('X2', 3.0, 4.0), ('X4', 2.0, 3.0), ('X5', 6.0, 2.0), ('X6', 6.0, 4.0), ('X7', 7.0, 3.0), ('X8', 7.0, 4.0), ('X9', 8.0, 5.0)]


### (2) Initialize with two centroids, (4, 6) and (5, 4). Use Euclidean distance as the distance metric.

In [8]:
#Eclidean Kmeans
#maxiter: control number of iteration
def Ekmeans(instances, k, maxiter, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        canvas = prepareWindow(instances)
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        paintClusters2D(canvas, clusters, centroids, "Initial centroids")
        time.sleep(delay)
    for itr in range(maxiter):
    #iteration = 0
    #while (centroids != prevCentroids):
        #iteration += 1
        clusters = assignAll(instances, centroids)
        
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids)
        
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["sse"]=withinss
    return result


In [9]:
initCentroids2 = [('',4,6),('',5,4)]
clustering = Ekmeans(dataset, 2, 1,animation=False, initCentroids=initCentroids1)
c=clustering["clusters"]
printTable(clustering["centroids"])
print('Xs assigned to two clusters are:','- cluster 1:', c[0], '- cluster 2:', c[1])

centroid0	2.50 6.50 
centroid1	5.75 3.88 
Xs assigned to two clusters are: - cluster 1: [('X1', 3.0, 5.0), ('X3', 2.0, 8.0)] - cluster 2: [('X2', 3.0, 4.0), ('X4', 2.0, 3.0), ('X5', 6.0, 2.0), ('X6', 6.0, 4.0), ('X7', 7.0, 3.0), ('X8', 7.0, 4.0), ('X9', 8.0, 5.0), ('X10', 7.0, 6.0)]


### (3) Initialize with two centroids, (3, 3) and (8, 3). Use Manhattan distance as the distance metric.


In [10]:
initCentroids3 = [('',3,3),('',8,3)]
clustering = Mkmeans(dataset, 2, 1,animation=False, initCentroids=initCentroids3)
c=clustering["clusters"]
printTable(clustering["centroids"])
print('Xs assigned to two clusters are:','- cluster 1:', c[0], '- cluster 2:', c[1])

centroid0	2.50 5.00 
centroid1	6.83 4.00 
Xs assigned to two clusters are: - cluster 1: [('X1', 3.0, 5.0), ('X2', 3.0, 4.0), ('X3', 2.0, 8.0), ('X4', 2.0, 3.0)] - cluster 2: [('X5', 6.0, 2.0), ('X6', 6.0, 4.0), ('X7', 7.0, 3.0), ('X8', 7.0, 4.0), ('X9', 8.0, 5.0), ('X10', 7.0, 6.0)]


### (4) Initialize with two centroids, (3, 2) and (4, 8). Use Manhattan distance as the distance metric.

In [11]:
initCentroids4 = [('',3,2),('',4,8)]
clustering = Mkmeans(dataset, 2, 1,animation=False, initCentroids=initCentroids4)
c=clustering["clusters"]
printTable(clustering["centroids"])
print('Xs assigned to two clusters are:','- cluster 1:', c[0], '- cluster 2:', c[1])

centroid0	4.86 3.57 
centroid1	5.67 6.33 
Xs assigned to two clusters are: - cluster 1: [('X1', 3.0, 5.0), ('X2', 3.0, 4.0), ('X4', 2.0, 3.0), ('X5', 6.0, 2.0), ('X6', 6.0, 4.0), ('X7', 7.0, 3.0), ('X8', 7.0, 4.0)] - cluster 2: [('X3', 2.0, 8.0), ('X9', 8.0, 5.0), ('X10', 7.0, 6.0)]


In [ ]:
data = pd.read_csv("C:/Work/Project/CAP5610/HW/HW5/Data.csv")

In [ ]:
data.info()

In [ ]:
data.rename(columns={'# wins in Season 2016 (x-axis)': 'x', '# wins in Season 2017 (y-axis)': 'y'}, inplace=True)

In [ ]:
data.columns = data.columns.str.strip()

In [ ]:
ax = data.set_index('x')['y'].plot(style='o')
def label_point(x, y, Team, ax):
    a = pd.concat({'x': x, 'y': y, 'Team': Team}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'], point['y'], str(point['Team']))

label_point(data.x, data.y, data.Team, ax)


In [16]:
data2 = pd.read_csv("C:/Work/Project/CAP5610/HW/HW5/iris.csv")
#Convert species to numeric
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data2['species'] = label_encoder.fit_transform(data2['species'])

    
Y=data2["species"]
X=data2.drop(["species"], axis=1)

In [17]:
#Read iris data
#df = pd.read_csv("C:/Work/Project/CAP5610/HW/HW5/iris.csv")
df = loadCSV("C:/Work/Project/CAP5610/HW/HW5/iris2.csv")
#showDataset2D(df)



## Task 2 K-Means Clustering with Real World Dataset 
### First, download the Iris data set from: https://archive.ics.uci.edu/ml/datasets/Iris. Then, implement the K-means algorithm. K-means algorithm computes the distance of a given data point pair. Replace the distance computation function with Euclidean distance, 1- Cosine similarity, and 1 – the Generalized Jarcard similarity
(https://www.itl.nist.gov/div898/software/dataplot/refman2/auxillar/jaccard.htm).
### Q1: Run K-means clustering with Euclidean, Cosine and Jarcard similarity. 
Specify K= the number of categorical values of y (the variable of label). Compare the SSEs of Euclidean-Kmeans
Cosine-K-means, Jarcard-K-means. Which method is better?

K=3; Euclidean metric is better.

In [28]:
#Cosine Similarity
def ckmeans(instances, k, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        canvas = prepareWindow(instances)
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        paintClusters2D(canvas, clusters, centroids, "Initial centroids")
        time.sleep(delay)
    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        clusters = cassignAll(instances, centroids)
        if animation:
            paintClusters2D(canvas, clusters, centroids, "Assign %d" % iteration)
            time.sleep(delay)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = ccomputeWithinss(clusters, centroids)
        if animation:
            paintClusters2D(canvas, clusters, centroids,
                            "Update %d, withinss %.1f" % (iteration, withinss))
            time.sleep(delay)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["sse"]= withinss
    result["iteration"] = iteration
    return result

#Jarcard Similarity
def jkmeans(instances, k, animation=False, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        canvas = prepareWindow(instances)
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        paintClusters2D(canvas, clusters, centroids, "Initial centroids")
        time.sleep(delay)
    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        clusters = assignAll(instances, centroids)
        if animation:
            paintClusters2D(canvas, clusters, centroids, "Assign %d" % iteration)
            time.sleep(delay)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = jcomputeWithinss(clusters, centroids)
        if animation:
            paintClusters2D(canvas, clusters, centroids,
                            "Update %d, withinss %.1f" % (iteration, withinss))
            time.sleep(delay)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["sse"]= withinss
    result["iteration"] = iteration
    return result


In [23]:
clusteringe = kmeans(df, 3, animation=False, initCentroids=None)
print('Euclidean Similarity SSE is: ', str(clusteringe["sse"]))

Euclidean Similarity SSE is:  47.95582906463267


In [ ]:
clusteringc = ckmeans(df, 3, animation=False, initCentroids=None)
print('Cosine Similarity SSE is: ', str(clusteringc["sse"]))

In [29]:
clusteringj = jkmeans(df, 3, animation=False, initCentroids=None)
print('Jaccard Similarity SSE is: ', str(clusteringj["sse"]))


Jaccard Similarity SSE is:  150.0


### Q2: Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. 
First,label each cluster with the label of the highest votes. Later, compute the accuracy of the Kmeans
with respect to the three similarity metrics. Which metric is better?

Euclidean metric is better with accuracy of 0.96.

In [31]:
#Models
from sklearn.neighbors import KNeighborsClassifier

#Model selection and evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn import metrics

    
knn = KNeighborsClassifier(n_neighbors = 3).fit(X, Y)
yhat = knn.predict(X)
print('Euclidean distance K-Means Accuracy: ', metrics.accuracy_score(Y, yhat))

Euclidean distance K-Means Accuracy:  0.96


In [ ]:
#Results dataframe
cols = ['Similarity','Euclidean-K-means','Cosine-K-means','Jarcard-K-means']

resul = pd.DataFrame(columns=cols)
resul.set_index("Similarity",inplace=True)
resul.loc['SSE'] = [0,0,0]
resul

In [ ]:
clusteringe = kmeans(df, 3, animation=False, initCentroids=None)
clusteringc = ckmeans(df, 3, animation=False, initCentroids=None)
clusteringj = jkmeans(df, 3, animation=False, initCentroids=None)
Similaritys = [clusteringe,clusteringc,clusteringj]
col = 0
for Similarity in Similaritys:
    clustering = Similarity
    resul.iloc[0,col] = clustering["sse"]
    col += 1
    
#printT(clustering["sse"])

### Q3: Which of Euclidean-K-means, Cosine-K-means, Jarcard-K-means requires more iterations and times?
Euclidean Kmeans requires more iterations and times.


In [32]:
%%time
clusteringe = kmeans(df, 3, animation=False, initCentroids=None)
print('Euclidean Similarity iterations: ', str(clusteringe["iteration"]))

Euclidean Similarity iterations:  6
Wall time: 8.67 ms


In [ ]:
%%time
clusteringc = ckmeans(df, 3, animation=False, initCentroids=None)
print('Cosine Similarity iterations: ', str(clusteringc["iteration"]))

In [33]:
%%time
clusteringj = jkmeans(df, 3, animation=False, initCentroids=None)
print('Jarcard Similarity iterations: ', str(clusteringj["iteration"]))

Jarcard Similarity iterations:  4
Wall time: 6.98 ms



### Q4: Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to the following three terminating conditions:
• when there is no change in centroid position
• when the SSE value increases in the next iteration
• when the maximum preset value (100) of iteration is complete
Which method requires more time or more iterations?

• when there is no change in centroid position : Cosine
• when the SSE value increases in the next iteration : Jarcard
• when the maximum preset value (100) of iteration is complete : Euclidean

### 3, Understanding K-Means:
### • Please give a scenario in which K-means cluster may not work very well?
K-Means may not work very well when the data have different density and overlap together. For example, when a denser cluster sits inside a less denser cluster.

#### • The classic K-means algorithm randomly initializes K centers. Is there any better strategy for selecting K initial centers?
Yes. For example Density-based initialization.

### 4, There are two clusters A (red) and B (blue), each has four members and plotted in Figure. 
The coordinates of each member are labeled in the figure. Compute the distance between two
clusters using Euclidean distance.
A.What is the distance between the two farthest members? (round to four decimal places here,
and next 2 problems);

(4.7, 3.2) and (6.2, 2.8) has the longest distance (SSE): 0.1600

B. What is the distance between the two closest members?

(5.0, 3.0) and (6.0, 3.0) has the closest distance (SSE): 0

C. What is the average distance between all pairs?

There are total 28 nothing duplicate pairs: average distance (SSE) is 0.004

D. Discuss which distance (A, B, C) is more robust to noises in this case?

(4.7, 3.2) (4.9, 3.1) and (5.0, 3.0) is more robust to noise because they have higher density together.

In [35]:
task4 = loadCSV("C:/Work/Project/CAP5610/HW/HW5/HW5task4.csv")

In [ ]:
task4

In [36]:
distance((4.7, 3.2),(6.2, 2.8))

0.16000000000000028

In [37]:
distance((5.0, 3.0),(6.0, 3.0))

0.0

In [38]:
distance(task4[0], task4[1])/28

0.0003571428571428578